# API access exercise

This script tests API requests for currency quotations and also exception handling using backoff as a function decorator

In [82]:
import requests
import json

In [7]:
url = 'https://economia.awesomeapi.com.br/json/last/USD-BRL'
ret = requests.get(url)

In [8]:
if ret:
    print(f"Requisição concluída com código {ret}")
    print(ret.text)
else:
    print(f"Requisição falhou! Código {ret}")

Requisição concluída com código <Response [200]>
{"USDBRL":{"code":"USD","codein":"BRL","name":"Dólar Americano/Real Brasileiro","high":"5.1167","low":"5.1166","varBid":"0.0004","pctChange":"0.01","bid":"5.1162","ask":"5.1171","timestamp":"1675120896","create_date":"2023-01-30 20:21:36"}}


In [9]:
json.loads(ret.text)

{'USDBRL': {'code': 'USD',
  'codein': 'BRL',
  'name': 'Dólar Americano/Real Brasileiro',
  'high': '5.1167',
  'low': '5.1166',
  'varBid': '0.0004',
  'pctChange': '0.01',
  'bid': '5.1162',
  'ask': '5.1171',
  'timestamp': '1675120896',
  'create_date': '2023-01-30 20:21:36'}}

In [28]:
def quote(value:float, pair:str):
    '''
        Converts a value in currency 1 to the currency 2 on the pair.
        :param value: Value in currency 1 to be converted
        :param pair: String containing a pair. Ex: USD-BRL (currency1-currency2)
    '''
    url = f'https://economia.awesomeapi.com.br/json/last/{pair}'
    ret = requests.get(url)
    currency_1 = json.loads(ret.text)[f'{pair.replace("-", "")}']
    print(f"{value} {pair[:3]} costs {float(currency_1['bid']) * value} {pair[4:]} today")

In [29]:
quote(20, 'USD-BRL')

20 USD costs 102.324 BRL today


In [30]:
quote(20, 'JPY-BRL')

20 JPY costs 0.7842 BRL today


In [31]:
quote(20, 'Arthur')

KeyError: 'Arthur'

You can treat exceptions

In [35]:
try:
    quote(20, 'Arthur')
except Exception as e:
    print(e)
else:
    print("OK")

'Arthur'


In [39]:
list_currencies_pairs = [
    "USD-BRL",
    "EUR-BRL",
    "BTC-BRL",
    "CRE-BRL" # republican credits to BRL
]

In [42]:
for currency_pair in list_currencies_pairs:
    try:
        quote(20, currency_pair)
    except:
        print(f"Failed to get pair: {currency_pair}")

20 USD costs 102.324 BRL today
20 EUR costs 110.99000000000001 BRL today
20 BTC costs 2341.54 BRL today
Failed to get pair: CRE-BRL


In [64]:
def multi_quote(value:float, pair_list:list):
    '''
        Converts a value in currency 1 to the currency 2 on the pair.
        :param value: Value in currency 1 to be converted
        :param pair_list: List of strings containing pairs. Ex: ["USD-BRL","EUR-BRL","BTC-BRL"] (currency1-currency2)
    '''
    for currency_pair in pair_list:
        try:
            quote(value, currency_pair)
        except:
            print(f"Failed to get pair: {currency_pair}")

In [45]:
multi_quote(20, list_currencies_pairs)

20 USD costs 102.324 BRL today
20 EUR costs 110.99000000000001 BRL today
20 BTC costs 2341.2799999999997 BRL today
Failed to get pair: CRE-BRL


Creating a decorator for error check

In [69]:
def error_check(func):
    def inner_func(*args, **kargs):
        try:
            func(*args, **kargs)
        except:
            print(f"{func.__name__} falhou")
    return inner_func

In [70]:
@error_check
def quote(value:float, pair:str):
    '''
        Converts a value in currency 1 to the currency 2 on the pair.
        :param value: Value in currency 1 to be converted
        :param pair: String containing a pair. Ex: USD-BRL (currency1-currency2)
    '''
    url = f'https://economia.awesomeapi.com.br/json/last/{pair}'
    ret = requests.get(url)
    currency_1 = json.loads(ret.text)[f'{pair.replace("-", "")}']
    print(f"{value} {pair[:3]} costs {float(currency_1['bid']) * value} {pair[4:]} today")

In [71]:
def multi_quote(value:float, pair_list:list):
    '''
        Converts a value in currency 1 to the currency 2 on the pair.
        :param value: Value in currency 1 to be converted
        :param pair_list: List of strings containing pairs. Ex: ["USD-BRL","EUR-BRL","BTC-BRL"] (currency1-currency2)
    '''
    for currency_pair in pair_list:
        quote(value, currency_pair)

In [72]:
multi_quote(20, list_currencies_pairs)

20 USD costs 102.324 BRL today
20 EUR costs 110.99000000000001 BRL today
20 BTC costs 2341.2799999999997 BRL today
quote falhou


Creating a decorator for treating several error could be time consuming. Module backoff has some helpful functions

In [74]:
import backoff

An example here is to demonstrate the usage of a backoff decorator

In [78]:
import random

#This receives a delay between tries, expo defines an exponential delay, then a list of exception triggers for backoff, then maximum tryouts.
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=10)
def test_func(*args, **kargs):
    rnd = random.random()
    print(f'''
        RND: {rnd}
        args: {args if args else "no args"}
        kargs: {kargs if kargs else "no kargs"}
    ''')

    if rnd < .2:
        raise ConnectionAbortedError('Connection terminated.')
    if rnd < .4:
        raise ConnectionRefusedError('Connection refused.')
    if rnd < .6:
        raise TimeoutError('Connection timeout.')
    else:
        return "OK!"

In [80]:
test_func()


        RND: 0.055056261541524876
        args: no args
        kargs: no kargs
    

        RND: 0.8620516551151075
        args: no args
        kargs: no kargs
    


'OK!'

In [81]:
test_func(10, name="testing")


        RND: 0.32805150754157386
        args: (10,)
        kargs: {'name': 'testing'}
    

        RND: 0.062475774108133986
        args: (10,)
        kargs: {'name': 'testing'}
    

        RND: 0.01583834679509677
        args: (10,)
        kargs: {'name': 'testing'}
    

        RND: 0.257117418255995
        args: (10,)
        kargs: {'name': 'testing'}
    

        RND: 0.2653838073519814
        args: (10,)
        kargs: {'name': 'testing'}
    

        RND: 0.8147351362213354
        args: (10,)
        kargs: {'name': 'testing'}
    


'OK!'